In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, MarkdownListOutputParser


## Especialista em direito

In [4]:
TEMPLATE = """
Você é um especialista jurídico e professor de direito no Brasil
Auxilia os alunos na criação de mapa mental (MindMap)
O mapa mental deve conter a ideia principal, ramos e subramos, sempre apontar os artigos 
Context:{input}
Formate a saida do mapa mental em mardown 
"""

llm = ChatOpenAI()

In [5]:
prompt = ChatPromptTemplate.from_template(template=TEMPLATE)
output_parser = StrOutputParser()
chain = prompt | llm | output_parser


In [6]:
res = chain.invoke({"input": "liste os diretiros fundamentais na constituição brasileira"})
print(res)

# Direitos Fundamentais na Constituição Brasileira

- **Ideia Principal:** Direitos Fundamentais na Constituição Brasileira
  
- **Ramos:**
  - **Direitos Individuais:**
    - Artigos: 5º ao 17º
  - **Direitos Coletivos:**
    - Artigos: 5º ao 8º
  - **Direitos Sociais:**
    - Artigos: 6º ao 11º
  - **Direitos de Nacionalidade:**
    - Artigos: 12º e 13º
  - **Direitos Políticos:**
    - Artigos: 14º ao 17º


## LLM memory

In [59]:
from langchain.chains import ConversationChain

# first initialize the large language model
llm = OpenAI(
	temperature=0.1,
)

# now initialize the conversation chain
conversation = ConversationChain(llm=llm) # A memoria é armazenada e acumulada em uma chave chamada history

    

In [8]:
#Template predefinido
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [61]:
conversation.invoke('Qual o meu nome')

{'input': 'Qual o meu nome',
 'history': 'Human: Eu sou fabio\nAI:  Olá Fabio! É um prazer conhecê-lo. Eu sou uma inteligência artificial criada para conversar com pessoas e aprender com elas. Como posso ajudá-lo hoje?',
 'response': ' Seu nome é Fabio. Eu sei disso porque você acabou de me dizer. Além disso, eu também tenho acesso a informações sobre você, como seu nome, idade, localização e interesses, que me ajudam a me comunicar melhor com você.'}

Usando **ConversationBufferMemory** , usamos muitos tokens muito rapidamente e até excedemos o limite da janela de contexto até mesmo dos LLMs mais avançados disponíveis hoje.

In [10]:
from langchain.chains.conversation.memory import ConversationBufferMemory

conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

conversation_buf("Good morning AI!")


{'input': 'Good morning AI!',
 'history': '',
 'response': " Good morning! It's a beautiful day today. The temperature is 72 degrees Fahrenheit and the sky is mostly clear with a few scattered clouds. How are you doing today?"}

Ao usar **ConversationSummaryMemory** , precisamos passar um LLM para o objeto porque o resumo é alimentado por um LLM. Podemos ver o prompt usado para fazer isso aqui:

In [11]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

conversation = ConversationChain(
	llm=llm,
	memory=ConversationSummaryMemory(llm=llm) # Necessário passar um llm, pois o resumo utiliza-o para sintetizar as conversas 
)

print(conversation.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


In [12]:
conversation('Em portugues')

{'input': 'Em portugues',
 'history': '',
 'response': ' Olá! Eu sou um assistente de inteligência artificial e posso falar português. Como posso ajudá-lo?'}

O **ConversationBufferWindowMemory** age da mesma forma que nossa “memória buffer” anterior , mas adiciona uma janela à memória. O que significa que mantemos apenas um determinado número de interações passadas antes de “esquecê- las”. Nós o usamos assim:

In [13]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

conversation = ConversationChain(
	llm=llm,
	memory=ConversationBufferWindowMemory(k=1) # k=1 — isso significa que a janela se lembrará da interação mais recente
)

O **ConversationSummaryBufferMemory** é uma mistura do ConversationSummaryMemory e do ConversationBufferWindowMemory . Ele resume as interações mais antigas em uma conversa, mantendo os max_token_limit tokens mais recentes em sua conversa. Ele é inicializado assim:

In [14]:
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory

conversation_sum_bufw = ConversationChain(
    llm=llm, memory=ConversationSummaryBufferMemory(
        llm=llm,
        max_token_limit=650
))

<img src='./assets/imgs/memoryBuffer.bmp' ></img>

### Memoria



In [174]:
from langchain_core.messages import HumanMessage, AIMessage 
from langchain_core.prompts import MessagesPlaceholder

llm = OpenAI(temperature=0.1)
llm = ChatOpenAI(temperature=0.1,)


prompt = ChatPromptTemplate.from_messages(
    [
        
     ('system', "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know."),
     MessagesPlaceholder(variable_name='chat_history'),
     ('human', "{input}"),
     ('ai',':')]
)

chat_history = [

]

def chat(llm, input, chat_history):
    response = llm.invoke({
        'input':input,
        'chat_history':chat_history
        }
    )
    return response

In [175]:
chain = prompt | llm | StrOutputParser()

In [176]:
user_input = 'Eu sou fabio'
# user_input = 'Qual o meu nome'
response = chat(chain, user_input, chat_history)
# chat_history.append(HumanMessage(content=user_input))
# chat_history.append(AIMessage(content=response))    
chat_history.append(['human',user_input])    
chat_history.append(['ai',response])    
print(response)
print(chat_history)


Olá, Fábio! Como posso te ajudar hoje?
[['human', 'Eu sou fabio'], ['ai', 'Olá, Fábio! Como posso te ajudar hoje?']]


In [177]:
user_input = 'Qual o meu nome'
response = chat(chain, user_input, chat_history)
# chat_history.append(HumanMessage(content=user_input))
# chat_history.append(AIMessage(content=response)) 
chat_history.append(['human',user_input])    
chat_history.append(['ai',response])  
# chat_history.append(f'{user_input} - {response}\n') 
print(response)
print(chat_history)


Desculpe, não tenho essa informação. Como posso te ajudar hoje?
[['human', 'Eu sou fabio'], ['ai', 'Olá, Fábio! Como posso te ajudar hoje?'], ['human', 'Qual o meu nome'], ['ai', 'Desculpe, não tenho essa informação. Como posso te ajudar hoje?']]


In [41]:
response

'Olá! Como posso te ajudar hoje?'

### Using ConversationBufferMemory from Langchain

In [93]:
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI, OpenAI
from langchain.chains import ConversationChain

chat = OpenAI(temperature=0)
template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.
Current conversation:
{history} 
Human: {input}
AI Assistant:"""

prompt = ChatPromptTemplate.from_template(template=template)
# prompt = prompt.invoke({'input':'','history':'Human: Qual o meu nome\nAI:  Seu nome é desconhecido para mim, pois sou um programa de computador e não tenho acesso a informações pessoais. Mas posso lhe chamar de "amigo" se preferir.\nHuman: Eu sou fabio\nAI:  Prazer em conhecê-lo, Fabio! Eu sou um assistente de inteligência artificial projetado para conversar e ajudar com informações. Como posso ser útil para você hoje?'})



memory = ConversationBufferMemory(size=5)  # sliding window buffer of size 5

buffer_chain = ConversationChain(
    prompt=prompt
    ,llm=chat, memory=memory)

In [92]:
buffer_chain.invoke(input='qual o meu nome')

ValueError: Got unsupported message type: Meu nome é fabio

In [58]:
prompt.format(input='',history='Meu nome é Antonella')

'Human: The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\nCurrent conversation:\nMeu nome é Antonella Meu nome é Fabio\nHuman: \nAI Assistant:'

### Automatic history management
https://python.langchain.com/v0.2/docs/tutorials/chatbot/

In [275]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


model = ChatOpenAI(model="gpt-3.5-turbo")

store = {}
chat_history = ChatMessageHistory()
chat_history.add_messages(['human','Ola, sou Fabio', 'human', 'minha blusa é azul'])

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = chat_history
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)


In [276]:
config = {"configurable": {"session_id": "abc2"}}

In [277]:
response = with_message_history.invoke(
    [HumanMessage(content="Qual o meu nome?")],
    config=config,
)

response.content    

'Você disse que se chama Fabio. Como posso ajudar você hoje?'

## Router chain 


[Router](https://medium.com/@gil.fernandes/langchains-router-chains-and-callbacks-722524c4aa42)

https://python.langchain.com/v0.1/docs/expression_language/how_to/routing/

Allow to dynamically select a pre-defined chain from a set of chains for a given input. In this blog we are going to explore

<img src='./assets/imgs/routerChain.bmp' width=900 ></img>

The following actions are executed here:

The user produces a text based input.  
The input is written to a file via a callback.  
The router selects the most appropriate chain from five options:  
- Python programmer
- Poet
- Wikipedia Expert
- Graphical artist
- UK, US Legal Expert   

The large language model responds.  
The output is again written to a file via a callback

In [10]:
from langchain_anthropic import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `LangChain`, `Anthropic`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain.invoke({"question": "how do I call Anthropic?"})

'Anthropic'

In [11]:
langchain_chain = PromptTemplate.from_template(
    """You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI()

anthropic_chain = PromptTemplate.from_template(
    """You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI()

general_chain = PromptTemplate.from_template(
    """Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI()

In [12]:
def route(info):
    if "anthropic" in info["topic"].lower():
        return anthropic_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain
    
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(route)

full_chain.invoke({"question": "how do I use Anthropic?"})

AIMessage(content="As Dario Amodei told me, to use Anthropic effectively, one must first understand the principles of anthropic reasoning and apply them to the specific context or problem at hand. This involves considering the observer's perspective, taking into account various possibilities and their probabilities, and making informed judgments based on these considerations. It is important to approach the use of anthropic reasoning with a critical and analytical mindset, ensuring that all relevant factors are taken into consideration before drawing conclusions.", response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 47, 'total_tokens': 141}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c2d8acb5-04ad-45ab-acbd-4a54ae025c3f-0', usage_metadata={'input_tokens': 47, 'output_tokens': 94, 'total_tokens': 141})

In [10]:
full_chain.invoke({"question": "how do I use LangChain?"})

AIMessage(content='As Harrison Chase told me, you can use LangChain by creating an account on the platform, uploading your content in the desired language, and selecting the language you want it translated into. The platform will then connect you with a qualified translator to complete the translation process efficiently and accurately.', response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 44, 'total_tokens': 100}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-255b3322-cb28-42d4-9460-4e6ab4f9bd09-0', usage_metadata={'input_tokens': 44, 'output_tokens': 56, 'total_tokens': 100})